## Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import sys

import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
chosen_stock = 'AAPL'

# Use yfinance to download the stock stock_data
stock_data = yf.download(chosen_stock, start='2015-01-01', end='2024-10-01', interval='1d')

# Display the first few rows of the stock_data
stock_data.tail()

In [ ]:
# Ensure the index is a DatetimeIndex
stock_data.index = pd.to_datetime(stock_data.index)


In [ ]:
# Set the frequency of the time series to 'B' (business day)
stock_data = stock_data.asfreq('B')

In [ ]:
missing_values = stock_data['Adj Close'].isna().sum()
print(f"Null values: {missing_values} \n \n")

In [ ]:
## Forward fill to handle missing stock_data points (optional, depending on the stock_data)
stock_data['Adj Close'].ffill(inplace= True)


missing_values = stock_data['Adj Close'].isna().sum()
print(f"Null values: {missing_values} \n \n")


In [ ]:
len(stock_data)

In [ ]:
# Adding indicators
stock_data['RSI']=ta.rsi(stock_data.Close, length=15)
stock_data['EMAF']=ta.ema(stock_data.Close, length=20)
stock_data['EMAM']=ta.ema(stock_data.Close, length=100)
stock_data['EMAS']=ta.ema(stock_data.Close, length=150)

# stock_data['Target'] = stock_data['Adj Close']-stock_data.Open
# stock_data['Target'] = stock_data['Target'].shift(-1)

# stock_data['TargetClass'] = [1 if stock_data.Target[i]>0 else 0 for i in range(len(stock_data))]

stock_data['TargetNextClose'] = stock_data['Adj Close'].shift(-1)

stock_data.dropna(inplace=True)
stock_data.reset_index(inplace = True)
stock_data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

In [ ]:
len(stock_data)

In [ ]:
stock_data.shape

In [ ]:
stock_data.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
stock_data_scaled= scaler.fit_transform(stock_data)
print(stock_data_scaled)

In [ ]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 30
print(stock_data_scaled.shape[0])
for j in range(8):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, stock_data_scaled.shape[0]):#backcandles+2
        X[j].append(stock_data_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(stock_data_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X)
print(X.shape)
print(y)
print(y.shape)

In [ ]:
df_scaled = pd.DataFrame(stock_data_scaled, columns=stock_data.columns)
df_scaled.head()

In [ ]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import keras

from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Input
from keras import optimizers

In [ ]:
lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse', metrics=["mse", "mae", keras.metrics.RootMeanSquaredError()])
history = model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)

In [ ]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'black', label = 'Test')
plt.plot(y_pred, color = 'green', label = 'pred')
plt.legend()
plt.show()